#  Look up wordnet antonyms for Yelp reviews

In [20]:
from nltk.tokenize import word_tokenize
import pymongo
from nltk.corpus import wordnet 

In [3]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient['yelp']

I have the yelp reviews and tips in one big collection called docs. If you uncomment the first line and comment the second, this should work for you. 

In [36]:
# random_review = mydb.reviews.find_one( { }, { '_id': 0, 'text': 1 })['text']
random_review = mydb.docs.find_one( { }, { '_id': 0, 'text': 1 })['text']

In [37]:
tokens = word_tokenize(random_review)

In [40]:
antonym_dict = dict()
for token in tokens:
    antonyms = []
    for syn in wordnet.synsets(token): 
        for l in syn.lemmas(): 
            if l.antonyms(): 
                for a in l.antonyms():
                    antonyms.append(a.name()) 
    if set(antonyms) != set():
        antonym_dict[token] = set(antonyms)
antonym_dict

{'Hard': {'easy', 'lightly', 'soft', 'voiced'},
 'new': {'old', 'worn'},
 'always': {'never'},
 'no': {'all', 'some', 'yes'},
 'stranger': {'acquaintance', 'familiar', 'native'},
 'chains': {'unchain'},
 'service': {'disservice'},
 'has': {'abstain', 'lack', 'refuse'},
 'taken': {'abstain', 'disclaim', 'give', 'obviate', 'refuse'},
 'whole': {'fractional', 'half', 'partly'},
 'level': {'raise'},
 'perfectly': {'imperfectly'},
 'green': {'ripe'},
 'black': {'white', 'whiten'},
 'comes': {'go', 'leave'},
 'most': {'fewest', 'least'},
 'relaxing': {'restless', 'stiffen', 'strain', 'tense'},
 'incredible': {'credible'},
 'get': {'end', 'leave', 'take_away'},
 'full': {'empty', 'thin', 'wane'},
 'head': {'foot', 'rear', 'tail'},
 'even': {'odd', 'uneven'},
 'worst': {'best', 'good', 'unregretful'},
 'scented': {'deodorize', 'deodourise', 'scentless'},
 'strong': {'impotent', 'weak'},
 'good': {'bad', 'badness', 'evil', 'evilness', 'ill'},
 'perfect': {'imperfect'},
 'was': {'differ'},
 'sta